<a href="https://colab.research.google.com/github/kobemawu/www/blob/master/LDA_TA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# nltkの文書群にgensimのLDAを適用してみる

## 準備

必要ライブラリ(全部pipで入ります)
 * nltk
 * gensim
 * pyLDAvis

In [ ]:
!pip install nltk
!pip install gensim
!pip install pyLDAvis

In [ ]:
#nltk使ったことない人は、pip install してから対話環境等で以下のdataset等をダウンロードしてください
import nltk
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("reuters")
nltk.download("punkt")

## データロード・前処理

In [3]:
#dataset読み込み
from nltk.corpus import reuters as corpus

### 今回はこういう文書(をBOW化したもの)を用います

In [ ]:
#必要に応じて以下のコードを実行してください．
!unzip /root/nltk_data/corpora/reuters.zip -d /root/nltk_data/corpora


In [7]:
for n,item in enumerate(corpus.words(corpus.fileids()[0])[:300]):
    print(item, end=" ")
    if (n%25) ==24:
      print(" ")

ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPAN RIFT Mounting trade friction between the U . S . And Japan has raised fears  
among many of Asia ' s exporting nations that the row could inflict far - reaching economic damage , businessmen and officials said . They  
told Reuter correspondents in Asian capitals a U . S . Move against Japan might boost protectionist sentiment in the U . S . And  
lead to curbs on American imports of their products . But some exporters said that while the conflict would hurt them in the long -  
run , in the short - term Tokyo ' s loss might be their gain . The U . S . Has said it will  
impose 300 mln dlrs of tariffs on imports of Japanese electronics goods on April 17 , in retaliation for Japan ' s alleged failure to  
stick to a pact not to sell semiconductors on world markets at below cost . Unofficial Japanese estimates put the impact of the tariffs at  
10 billion dlrs and spokesmen for major electronics firms said they would virtually halt exports 

In [8]:
#全document数
len(corpus.fileids())

10788

In [9]:

#前からk個のdocumentのみで学習する場合
#k=1000
#docs=[corpus.words(fileid) for fileid in corpus.fileids()[:k]]

#全documentで学習する場合
docs=[corpus.words(fileid) for fileid in corpus.fileids()]

print(docs[:5])
print("num of docs:", len(docs))

[['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', ...], ['CHINA', 'DAILY', 'SAYS', 'VERMIN', 'EAT', '7', '-', ...], ['JAPAN', 'TO', 'REVISE', 'LONG', '-', 'TERM', ...], ['THAI', 'TRADE', 'DEFICIT', 'WIDENS', 'IN', 'FIRST', ...], ['INDONESIA', 'SEES', 'CPO', 'PRICE', 'RISING', ...]]
num of docs: 10788


## 前処理 

In [10]:
#ストップワードリストの作成

#1 nltkのストップワードリスト
en_stop = nltk.corpus.stopwords.words('english')

# 一度LDAしてみる等して，適宜ノイズになってそうな記号等を見つけて，ストップワードリストに新たに加える
#【発展】記号や数字は正規表現で消してみましょう
en_stop= ["``","/",",.",".,",";","--",":",")","(",'"','&',"'",'),',',"','-','.,','.,"','.-',"?",">","<"]                  \
         +["0","1","2","3","4","5","6","7","8","9","10","11","12","86","1986","1987","000"]                                                      \
         +["said","say","u","v","mln","ct","net","dlrs","tonne","pct","shr","nil","company","lt","share","year","billion","price"]          \
         +en_stop

In [11]:
#前処理関数の作成

from nltk.corpus import wordnet as wn #lemmatize関数のためのimport

def preprocess_word(word, stopwordset):
    
    #1.make words lower  example: Python =>python
    word=word.lower()
    
    #2.remove "," and "."
    if word in [",","."]:
        return None
    
    #3.remove stopword  example: the => (None) 
    if word in stopwordset:
        return None
    
    #4.lemmatize  example: cooked=>cook
    lemma = wn.morphy(word)
    if lemma is None:
        return word

    elif lemma in stopwordset: #lemmatizeしたものがstopwordである可能性がある
        return None
    else:
        return lemma
    

def preprocess_document(document):
    document=[preprocess_word(w, en_stop) for w in document]
    document=[w for w in document if w is not None]
    return document

def preprocess_documents(documents):
    return [preprocess_document(document) for document in documents]

In [12]:
#before
print(docs[0][:25]) 

#after
print(preprocess_documents(docs)[0][:25])

['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears']
['asian', 'exporter', 'fear', 'damage', 'japan', 'rift', 'mounting', 'trade', 'friction', 'japan', 'raise', 'fear', 'among', 'many', 'asia', 'exporting', 'nation', 'row', 'could', 'inflict', 'far', 'reaching', 'economic', 'damage', 'businessmen']


## LDA準備

In [13]:
import gensim
from gensim import corpora

In [14]:
#documentを，gensim LDAが読み込めるデータ構造にする

#辞書の作成
dictionary = corpora.Dictionary(preprocess_documents(docs))
#コーパスの作成
corpus_ = [dictionary.doc2bow(doc) for doc in preprocess_documents(docs)]

In [ ]:
#Dictionary:gensimにおける辞書クラス
#token2id属性には単語と辞書IDとの対応が格納される

print(dictionary.token2id)

In [16]:
#corpusにはdocumentごとに単語の(ID、出現回数)のリストが得られる

print(corpus_[0][:10]) #文章での出現順でなく辞書IDの若い順なことに注意

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 3), (8, 1), (9, 1)]


In [17]:
#before
print([w.lower() for w in corpus.sents(corpus.fileids()[0])[0]])

#after
print(dictionary.doc2bow([w.lower() for w in corpus.sents(corpus.fileids()[0])[0]]))

#これを全文書の全文に適用したのがcorpus_

['asian', 'exporters', 'fear', 'damage', 'from', 'u', '.', 's', '.-', 'japan', 'rift', 'mounting', 'trade', 'friction', 'between', 'the', 'u', '.', 's', '.', 'and', 'japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.']
[(16, 1), (19, 1), (20, 1), (37, 1), (55, 1), (59, 2), (72, 1), (85, 1), (88, 1), (89, 1), (96, 1), (116, 1), (120, 2), (142, 1), (157, 1), (198, 1), (209, 1), (210, 1), (256, 1)]


## LDA学習

In [18]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=20,
                                           id2word=dictionary,
                                           alpha=0.1,                             #optional LDAのハイパーパラメータalpha
                                           eta=0.1,                                 #optional LDAのハイパーパラメータbeta
                                           #minimum_probability=0.0    #optional 学習結果に格納するトピック・単語の生起確率の下限
                                          )

## パラメータの確認

In [19]:
#(トピックID, 当該トピックにおける単語とそのprobability)  ※　のうち、上位num_words位 

topics = ldamodel.print_topics(num_words=15)
for topic in topics:
    print(topic)

(0, '0.021*"sales" + 0.018*"qtr" + 0.014*"4th" + 0.013*"note" + 0.010*"rev" + 0.009*"1985" + 0.009*"include" + 0.009*"loss" + 0.008*"january" + 0.007*"inc" + 0.007*"gain" + 0.007*"end" + 0.007*"avg" + 0.007*"shrs" + 0.007*"28"')
(1, '0.014*"would" + 0.010*"ec" + 0.006*"commission" + 0.006*"gencorp" + 0.005*"plan" + 0.005*"european" + 0.005*"market" + 0.005*"tender" + 0.005*"export" + 0.004*"analyst" + 0.004*"one" + 0.004*"last" + 0.004*"also" + 0.004*"tax" + 0.004*"per"')
(2, '0.049*"stg" + 0.023*"bank" + 0.021*"rev" + 0.020*"shrs" + 0.019*"qtr" + 0.019*"avg" + 0.015*"market" + 0.014*"jan" + 0.012*"1st" + 0.011*"k" + 0.011*"money" + 0.010*"31" + 0.009*"16" + 0.009*"corp" + 0.009*"bill"')
(3, '0.012*"gulf" + 0.011*"corp" + 0.008*"common" + 0.008*"merger" + 0.007*"inc" + 0.007*"iran" + 0.006*"stock" + 0.006*"oil" + 0.005*"pacific" + 0.005*"missile" + 0.005*"ship" + 0.005*"pipeline" + 0.005*"shareholder" + 0.005*"shipping" + 0.004*"would"')
(4, '0.020*"oper" + 0.018*"april" + 0.018*"recor

In [20]:
#[(当該documentにおけるトピックIDとそのprobability　)]　 ※　のうち、minimum_probabilityの値を超えるもの

for n,item in enumerate(corpus_[:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel.get_document_topics(item))

document ID 0:[(1, 0.026368447), (6, 0.07723138), (7, 0.021549119), (8, 0.12972574), (10, 0.02202973), (12, 0.6120421), (17, 0.10790494)]
document ID 1:[(5, 0.20143344), (6, 0.07478321), (8, 0.162274), (16, 0.53141564)]
document ID 2:[(12, 0.19364278), (14, 0.7057467), (16, 0.0545798), (17, 0.03174169)]
document ID 3:[(7, 0.046529885), (13, 0.12958449), (16, 0.46027732), (17, 0.34658366)]
document ID 4:[(1, 0.02952471), (8, 0.122700825), (9, 0.015160659), (16, 0.8166106)]
document ID 5:[(7, 0.017792517), (8, 0.45537633), (14, 0.51178426)]
document ID 6:[(0, 0.013757105), (1, 0.05697171), (7, 0.0660678), (8, 0.6776655), (14, 0.018788645), (16, 0.12739612), (17, 0.033866905)]
document ID 7:[(5, 0.14274766), (6, 0.29586512), (7, 0.1046729), (13, 0.13140224), (16, 0.27842695)]
document ID 8:[(1, 0.109378986), (4, 0.05232196), (5, 0.019391814), (7, 0.7005131), (9, 0.07516294), (11, 0.02322761)]
document ID 9:[(1, 0.13863876), (6, 0.018394459), (8, 0.015829975), (10, 0.23320936), (12, 0.5241

In [21]:
#documentのcategory
categories=[corpus.categories(fileid) for fileid in corpus.fileids()]

In [22]:
n=0

#n番目のdocumentのトピック分布
print(ldamodel.get_document_topics(corpus_[n]))

#n番目のdocumentのcategory
print(categories[n])

#n番目のdocumentの生の文章
print(" ".join(docs[n]))

[(1, 0.026358789), (6, 0.07723607), (7, 0.021553531), (8, 0.1297238), (10, 0.022026742), (12, 0.6120462), (17, 0.107906334)]
['trade']
ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPAN RIFT Mounting trade friction between the U . S . And Japan has raised fears among many of Asia ' s exporting nations that the row could inflict far - reaching economic damage , businessmen and officials said . They told Reuter correspondents in Asian capitals a U . S . Move against Japan might boost protectionist sentiment in the U . S . And lead to curbs on American imports of their products . But some exporters said that while the conflict would hurt them in the long - run , in the short - term Tokyo ' s loss might be their gain . The U . S . Has said it will impose 300 mln dlrs of tariffs on imports of Japanese electronics goods on April 17 , in retaliation for Japan ' s alleged failure to stick to a pact not to sell semiconductors on world markets at below cost . Unofficial Japanese estimates put the i

## 可視化

In [23]:
#import pyLDAvis.gensim
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [24]:
#全documentを学習に用いた場合結構時間がかかる(20min~)
#gensimではK個のトピックに0~K-1のidが割り振られていたのに対し，pyLDAvisでは1~Kのidが割り振られていることに注意

lda_display = pyLDAvis.gensim_models.prepare(ldamodel, corpus_, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [25]:
#上で可視化したモデルをgoogle drive上にsaveできる

pyLDAvis.save_html(lda_display,'vis.html')

In [26]:
!ls

sample_data  vis.html
